In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}





Preconfiguring packages ...
Selecting previously unselected package cron.
(Reading database ... 18408 files and directories currently installed.)
Preparing to unpack .../00-cron_3.0pl1-128ubuntu5_amd64.deb ...
Unpacking cron (3.0pl1-128ubuntu5) ...
Selecting previously unselected package libapparmor1:amd64.
Preparing to unpack .../01-libapparmor1_2.11.0-2ubuntu17.1_amd64.deb ...
Unpacking libapparmor1:amd64 (2.11.0-2ubuntu17.1) ...
Selecting previously unselected package libdbus-1-3:amd64.
Preparing to unpack .../02-libdbus-1-3_1.10.22-1ubuntu1_amd64.deb ...
Unpacking libdbus-1-3:amd64 (1.10.22-1ubuntu1) ...
Selecting previously unselected package dbus.
Preparing to unpack .../03-dbus_1.10.22-1ubuntu1_amd64.deb ...
Unpacking dbus (1.10.22-1ubuntu1) ...
Selecting previously unselected package dirmngr.
Preparing to unpack .../04-dirmngr_2.1.15-1ubuntu8.1_amd64.deb ...
Unpacking dirmngr (2.1.15-1ubuntu8.1) ...
Selecting previously unselected package distro-info-data.
Preparing to unpack .

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
cd drive/project_data/

/content/drive/project_data


In [0]:
import keras
from keras import backend as K
import tensorflow as tf
import tensorflow
from keras.models import Model
from keras import layers
from keras.models import Sequential
from keras.layers import Reshape, Activation, Conv2D, Input, MaxPooling2D, BatchNormalization, Flatten, Dense, Lambda
from keras.layers.advanced_activations import LeakyReLU, Softmax
from keras.layers.merge import concatenate
from keras.optimizers import SGD, Adam, RMSprop
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard

import numpy as np
import os
import cv2
import math

Using TensorFlow backend.


In [0]:
X = np.load('final_pascal_npy_datas/2007_test.npy')
Y = np.load('fixed_labels/2007_test_labels.npy')

In [0]:
print(X.shape)
Y.shape

(4952, 300, 300, 3)


(4952, 18, 18, 20)

In [0]:
def ultimate_loss(y_pred, y_true):
    # shape - (batch_size, 38, 38, 20)
    y_pred = tf.reshape(y_pred, [-1,18*18,20])
    y_true = tf.reshape(y_true, [-1,18*18,20])
    loss = tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_true,logits=y_pred)
    return loss

In [0]:
input_image = Input(shape=(300, 300, 3)) 
#input_image = tf.convert_to_tensor(np.zeros(shape=(300,300,3)))

#############################################################
####### VGG16 Model  ########################################
#############################################################
# Block 1
net = layers.Conv2D(64, (3, 3),
                  activation='relu',
                  padding='same',
                  name='block1_conv1', trainable=False)(input_image)
net = BatchNormalization()(net)
net = layers.Conv2D(64, (3, 3),
                  activation='relu',
                  padding='same',
                  name='block1_conv2', trainable=False)(net)
net = BatchNormalization()(net)
net = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(net)

# Block 2
net = layers.Conv2D(128, (3, 3),
                  activation='relu',
                  padding='same',
                  name='block2_conv1', trainable=False)(net)
net = BatchNormalization()(net)
net = layers.Conv2D(128, (3, 3),
                  activation='relu',
                  padding='same',
                  name='block2_conv2', trainable=False)(net)
net = BatchNormalization()(net)
net = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(net)

# Block 3
net = layers.Conv2D(256, (3, 3),
                  activation='relu',
                  padding='same',
                  name='block3_conv1', trainable=False)(net)
net = BatchNormalization()(net)
net = layers.Conv2D(256, (3, 3),
                  activation='relu',
                  padding='same',
                  name='block3_conv2')(net)
net = BatchNormalization()(net)
net = layers.Conv2D(256, (3, 3),
                  activation='relu',
                  padding='same',
                  name='block3_conv3', trainable=False)(net)
net = BatchNormalization()(net)
net = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(net)

# Block 4
net = layers.Conv2D(512, (3, 3),
                  activation='relu',
                  padding='same',
                  name='block4_conv1', trainable=False)(net)
net = BatchNormalization()(net)
net = layers.Conv2D(512, (3, 3),
                  activation='relu',
                  padding='same',
                  name='block4_conv2', trainable=False)(net)
net = BatchNormalization()(net)
net = layers.Conv2D(512, (3, 3),
                  activation='relu',
                  padding='same',
                  name='block4_conv3', trainable=False)(net)
net = BatchNormalization()(net)
net = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(net)

# Block 5
net = layers.Conv2D(512, (3, 3),
                  activation='relu',
                  padding='same',
                  name='block5_conv1', trainable=False)(net)
net = BatchNormalization()(net)
net = layers.Conv2D(512, (3, 3),
                  activation='relu',
                  padding='same',
                  name='block5_conv2', trainable=False)(net)
net = BatchNormalization()(net)
net = layers.Conv2D(512, (3, 3),
                  activation='relu',
                  padding='same',
                  name='block5_conv3', trainable=False)(net)
net = BatchNormalization()(net)


net = layers.Conv2D(100, (3, 3),
                  activation='relu',
                  padding='same',
                  name='blockA_conv1')(net)

net = layers.Conv2D(50, (3, 3),
                  activation='relu',
                  padding='same',
                  name='blockA_conv2')(net)

net = layers.Conv2D(20, (3, 3),
                  activation='softmax',
                  padding='same',
                  name='blockA_conv3')(net)




In [0]:
model = Model(input_image, net)

In [0]:
model.load_weights('classifier_august/Classifier20epoch.h5',by_name=True)

In [0]:
optimizer = SGD(lr=0.001, decay=0.0, momentum=0.9)

model.compile(loss=ultimate_loss, optimizer=optimizer, metrics=['accuracy'])



In [0]:
model.fit(X, Y, batch_size=16, epochs=1)

Epoch 1/1
 160/4952 [..............................] - ETA: 4:39 - loss: 2.9959 - acc: 0.0284